In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.simplefilter('ignore')
import joblib

In [2]:
#read in cleaned csv file
df = pd.read_csv('data_clean.csv')

In [3]:
df.head()

,Unnamed: 0,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [4]:
#review possible y values
df['koi_disposition'].unique()

array(['CONFIRMED', 'FALSE POSITIVE', 'CANDIDATE'], dtype=object)

In [5]:
#get y values into new df
dfy_ = df['koi_disposition']
#reshape df with only y values
dfy_ = dfy_.values.reshape(-1, 1)
label = LabelEncoder()
#transform y values into format in line with what a ML model can use
dfy = label.fit(dfy_).transform(dfy_)


In [6]:
#get unique values of possible y values into a variable 
y_labels = label.classes_


In [7]:
#create df with possible y values in each row
y_labels_df = pd.DataFrame({'label': y_labels})
y_labels_df

,label
0,CANDIDATE
1,CONFIRMED
2,FALSE POSITIVE


In [8]:
#drop extra column with index value
df = df.drop(columns=['Unnamed: 0'])
#todo - don't save the index on the to_csv function in the cleaning notebook
#will add a note that this should have been done

In [9]:
#drop y values from df containing only x values
dfx = df.drop(columns=['koi_disposition'])

In [10]:
dfx.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [11]:
print(dfx.shape, dfy.shape)
#if the y shape causes problems, I can put into a DF and reshape

(6991, 40) (6991,)


In [12]:
#split data into training and testing data
x_train, x_test, y_train, y_test = train_test_split(dfx, dfy, random_state=55)
#scale x values
x_scaler = StandardScaler().fit(x_train)
x_train = x_scaler.transform(x_train)
x_test = x_scaler.transform(x_test)

In [13]:
model = RandomForestClassifier(n_estimators=200)


In [14]:
model.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [15]:
#score the model with test data
model.score(x_test, y_test)

0.8895881006864989

In [16]:
#see the score with training data
model.score(x_train, y_train)

1.0

In [17]:
#predicts the koi_disposition of the 354 index of the df
res = model.predict(dfx.loc[354,].values.reshape(1, -1))[0]
#look up the corresponding string classification associated with the numerical classification
y_labels_df.loc[res,]['label']

'FALSE POSITIVE'

In [18]:
#get value of each feature in a list
feature_value = list(model.feature_importances_)
#list of feature names
feature_names = list(dfx.columns)

In [19]:
#loop to zip the feature name and its value together
features_and_values = [(feature, importance) for feature, importance in zip(feature_names, feature_value)]
features_and_values

[('koi_fpflag_nt', 0.10169284863738372),
 ('koi_fpflag_ss', 0.06159437691890167),
 ('koi_fpflag_co', 0.10743682168342482),
 ('koi_fpflag_ec', 0.03600673430965329),
 ('koi_period', 0.022491656942975946),
 ('koi_period_err1', 0.017157148305041683),
 ('koi_period_err2', 0.01708880117213136),
 ('koi_time0bk', 0.013208570587122258),
 ('koi_time0bk_err1', 0.025066513841492746),
 ('koi_time0bk_err2', 0.026321670128889342),
 ('koi_impact', 0.017313195910416097),
 ('koi_impact_err1', 0.010848925878824559),
 ('koi_impact_err2', 0.010581850521044895),
 ('koi_duration', 0.023194381372630874),
 ('koi_duration_err1', 0.03151037857381347),
 ('koi_duration_err2', 0.033976468008043464),
 ('koi_depth', 0.022228818394139028),
 ('koi_depth_err1', 0.013675761590655892),
 ('koi_depth_err2', 0.014475978052206195),
 ('koi_prad', 0.048673275334492),
 ('koi_prad_err1', 0.03558425727326433),
 ('koi_prad_err2', 0.03540801047183052),
 ('koi_teq', 0.01575718276870175),
 ('koi_insol', 0.016929084726568698),
 ('koi_i

In [20]:
sorted(features_and_values, key = lambda x: x[1], reverse = True) 

[('koi_fpflag_co', 0.10743682168342482),
 ('koi_fpflag_nt', 0.10169284863738372),
 ('koi_fpflag_ss', 0.06159437691890167),
 ('koi_model_snr', 0.05008000632258233),
 ('koi_prad', 0.048673275334492),
 ('koi_fpflag_ec', 0.03600673430965329),
 ('koi_prad_err1', 0.03558425727326433),
 ('koi_prad_err2', 0.03540801047183052),
 ('koi_duration_err2', 0.033976468008043464),
 ('koi_duration_err1', 0.03151037857381347),
 ('koi_steff_err2', 0.026786539846080103),
 ('koi_time0bk_err2', 0.026321670128889342),
 ('koi_steff_err1', 0.025733156686602),
 ('koi_time0bk_err1', 0.025066513841492746),
 ('koi_duration', 0.023194381372630874),
 ('koi_period', 0.022491656942975946),
 ('koi_depth', 0.022228818394139028),
 ('koi_insol_err1', 0.019060070268834405),
 ('koi_impact', 0.017313195910416097),
 ('koi_period_err1', 0.017157148305041683),
 ('koi_period_err2', 0.01708880117213136),
 ('koi_insol', 0.016929084726568698),
 ('koi_teq', 0.01575718276870175),
 ('koi_insol_err2', 0.015003500058902488),
 ('koi_depth

In [21]:
#create a copy of dfx
dfx_important_features = dfx.copy()
#drop columns of features with importance value of less than 1%
dfx_important_features = dfx_important_features.drop(columns=['koi_slogg', 'koi_srad', 'koi_steff', 'koi_slogg_err1',
                                                             'koi_srad_err2', 'koi_tce_plnt_num'])
#the total significience of dropped columns is just under 5%

In [22]:
#split data into testing and training data
#the random state remained at 55

x_train, x_test, y_train, y_test = train_test_split(dfx_important_features, dfy, random_state=55)
#scale x values
x_scaler = StandardScaler().fit(x_train)
x_train = x_scaler.transform(x_train)
x_test = x_scaler.transform(x_test)

In [23]:
#create new model
model_important_features = RandomForestClassifier(n_estimators=200)


In [24]:
#train new model
model_important_features.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=200,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [25]:
#score new model with testing data
model_important_features.score(x_test, y_test)
#didn't help very much, the r^2 only went up by about 0.003

0.8913043478260869

In [26]:
model_important_features.score(x_train, y_train)

1.0

In [27]:
max_depth = list(np.arange(20, 71, 10))
max_features = list(np.arange(5, 26, 5))
max_features.append('auto')
max_features.append('log2')
max_depth.append(None)
param_grid = {
    'bootstrap': [True, False], 
    'n_estimators': [250],
    'max_features': max_features,
    'max_depth': max_depth,
    'n_jobs': [-1]
}
grid = GridSearchCV(model_important_features, param_grid, verbose=3)

In [28]:
grid.fit(x_train, y_train)

Fitting 3 folds for each of 98 candidates, totalling 294 fits
[CV] bootstrap=True, max_depth=20, max_features=5, n_estimators=250, n_jobs=-1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=True, max_depth=20, max_features=5, n_estimators=250, n_jobs=-1, score=0.890, total=   2.4s
[CV] bootstrap=True, max_depth=20, max_features=5, n_estimators=250, n_jobs=-1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s remaining:    0.0s


[CV]  bootstrap=True, max_depth=20, max_features=5, n_estimators=250, n_jobs=-1, score=0.890, total=   0.7s
[CV] bootstrap=True, max_depth=20, max_features=5, n_estimators=250, n_jobs=-1 


...

[CV]  bootstrap=False, max_depth=30, max_features=15, n_estimators=250, n_jobs=-1, score=0.895, total=   2.2s
[CV] bootstrap=False, max_depth=30, max_features=15, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=30, max_features=15, n_estimators=250, n_jobs=-1, score=0.896, total=   2.4s
[CV] bootstrap=False, max_depth=30, max_features=15, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=30, max_features=15, n_estimators=250, n_jobs=-1, score=0.887, total=   2.2s
[CV] bootstrap=False, max_depth=30, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=30, max_features=20, n_estimators=250, n_jobs=-1, score=0.899, total=   2.8s
[CV] bootstrap=False, max_depth=30, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=30, max_features=20, n_estimators=250, n_jobs=-1, score=0.892, total=   3.0s
[CV] bootstrap=False, max_depth=30, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=30, max

[CV]  bootstrap=False, max_depth=50, max_features=15, n_estimators=250, n_jobs=-1, score=0.893, total=   2.2s
[CV] bootstrap=False, max_depth=50, max_features=15, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=50, max_features=15, n_estimators=250, n_jobs=-1, score=0.886, total=   2.4s
[CV] bootstrap=False, max_depth=50, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=50, max_features=20, n_estimators=250, n_jobs=-1, score=0.897, total=   3.0s
[CV] bootstrap=False, max_depth=50, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=50, max_features=20, n_estimators=250, n_jobs=-1, score=0.892, total=   3.4s
[CV] bootstrap=False, max_depth=50, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=50, max_features=20, n_estimators=250, n_jobs=-1, score=0.886, total=   3.3s
[CV] bootstrap=False, max_depth=50, max_features=25, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=50, max

[CV]  bootstrap=False, max_depth=70, max_features=15, n_estimators=250, n_jobs=-1, score=0.888, total=   2.2s
[CV] bootstrap=False, max_depth=70, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=70, max_features=20, n_estimators=250, n_jobs=-1, score=0.899, total=   2.7s
[CV] bootstrap=False, max_depth=70, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=70, max_features=20, n_estimators=250, n_jobs=-1, score=0.892, total=   2.9s
[CV] bootstrap=False, max_depth=70, max_features=20, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=70, max_features=20, n_estimators=250, n_jobs=-1, score=0.883, total=   2.9s
[CV] bootstrap=False, max_depth=70, max_features=25, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=70, max_features=25, n_estimators=250, n_jobs=-1, score=0.889, total=   3.6s
[CV] bootstrap=False, max_depth=70, max_features=25, n_estimators=250, n_jobs=-1 
[CV]  bootstrap=False, max_depth=70, max

[Parallel(n_jobs=1)]: Done 294 out of 294 | elapsed:  7.9min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=200, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             i

In [29]:
best_parameters = grid.best_params_
best_parameters

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 25,
 'n_estimators': 250,
 'n_jobs': -1}

In [30]:
#create 'best' model after hypertuning
#also experiment with some of the 'high scoring' grid models, and with the n_estimators
model_best = RandomForestClassifier(bootstrap=True, max_depth=60, max_features=25, n_estimators=250)

In [31]:
#fit model
model_best.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=60, max_features=25, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [32]:
#score model
model_best.score(x_test, y_test)
#score did not change much

0.8913043478260869

In [33]:
#score with training data
model_best.score(x_train, y_train)

1.0

In [34]:
#try second grid search to tune the number of estimators 
param_grid = {'n_estimators': list(np.arange(200,451,5)),
              'bootstrap': [True], 
              'max_depth': [60],
              'max_features': [25],
              'n_jobs': [-1]  #use all cores to speed up the grid search
             }
grid = GridSearchCV(model_important_features, param_grid, verbose=3)

In [35]:
grid.fit(x_train, y_train)

Fitting 3 folds for each of 51 candidates, totalling 153 fits
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=200, n_jobs=-1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=200, n_jobs=-1, score=0.902, total=   1.7s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=200, n_jobs=-1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s


[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=200, n_jobs=-1, score=0.894, total=   1.6s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=200, n_jobs=-1 


...

[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=275, n_jobs=-1, score=0.898, total=   2.8s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=275, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=275, n_jobs=-1, score=0.887, total=   2.5s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=280, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=280, n_jobs=-1, score=0.901, total=   2.7s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=280, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=280, n_jobs=-1, score=0.892, total=   2.6s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=280, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=280, n_jobs=-1, score=0.887, total=   2.7s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=285, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=2

[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=350, n_jobs=-1, score=0.903, total=   2.9s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=350, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=350, n_jobs=-1, score=0.894, total=   3.1s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=350, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=350, n_jobs=-1, score=0.892, total=   3.2s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=355, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=355, n_jobs=-1, score=0.902, total=   3.3s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=355, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=355, n_jobs=-1, score=0.895, total=   3.8s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=355, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=2

[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=420, n_jobs=-1, score=0.891, total=   3.9s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=425, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=425, n_jobs=-1, score=0.902, total=   3.6s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=425, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=425, n_jobs=-1, score=0.897, total=   3.6s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=425, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=425, n_jobs=-1, score=0.890, total=   3.8s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=430, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=25, n_estimators=430, n_jobs=-1, score=0.901, total=   3.7s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=430, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=2

[Parallel(n_jobs=1)]: Done 153 out of 153 | elapsed:  7.6min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=200, n_jobs=None,
                                              oob_score=...
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid=

In [36]:
best_parameters1 = grid.best_params_
best_parameters1

{'bootstrap': True,
 'max_depth': 60,
 'max_features': 25,
 'n_estimators': 375,
 'n_jobs': -1}

In [37]:
#experiment with some of the top N-estimators from above grid search (top result did not yield improvement)
model_best1 = RandomForestClassifier(bootstrap=True, max_depth=60, max_features=25, n_estimators=375)

In [38]:
#fit model
model_best1.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=60, max_features=25, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=375,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [39]:
#score model
model_best1.score(x_test, y_test)

0.8930205949656751

In [40]:
#score with training data
model_best1.score(x_train, y_train)

1.0

In [41]:
#removing 1-4% of unimportant values to further refine what the model looks at
#get value of each feature in a list
feature_value2 = list(model_best1.feature_importances_)
#list of feature names (from most recent model)
feature_names2 = list(dfx_important_features.columns)

In [42]:
features_and_values2 = [(feature, importance) for feature, importance in zip(feature_names2, feature_value2)]
features_and_values2

[('koi_fpflag_nt', 0.1944278724588487),
 ('koi_fpflag_ss', 0.15332383613745862),
 ('koi_fpflag_co', 0.16990271768848692),
 ('koi_fpflag_ec', 0.038037298989236794),
 ('koi_period', 0.011500573879916594),
 ('koi_period_err1', 0.0060245886560102),
 ('koi_period_err2', 0.006148360591268294),
 ('koi_time0bk', 0.012516951964590102),
 ('koi_time0bk_err1', 0.008263757749428616),
 ('koi_time0bk_err2', 0.008402811703359315),
 ('koi_impact', 0.017331343464750387),
 ('koi_impact_err1', 0.008325531047207688),
 ('koi_impact_err2', 0.008775394725799426),
 ('koi_duration', 0.020348805604299632),
 ('koi_duration_err1', 0.02265339823941591),
 ('koi_duration_err2', 0.020170288658895214),
 ('koi_depth', 0.008125982507724205),
 ('koi_depth_err1', 0.007924008253270708),
 ('koi_depth_err2', 0.00785384254385342),
 ('koi_prad', 0.035252804830769244),
 ('koi_prad_err1', 0.009034673029845708),
 ('koi_prad_err2', 0.016008810965106368),
 ('koi_teq', 0.004460961279339532),
 ('koi_insol', 0.004530137523258569),
 ('k

In [43]:
sorted(features_and_values2, key = lambda x: x[1], reverse = True) 

[('koi_fpflag_nt', 0.1944278724588487),
 ('koi_fpflag_co', 0.16990271768848692),
 ('koi_fpflag_ss', 0.15332383613745862),
 ('koi_model_snr', 0.1088328079950184),
 ('koi_fpflag_ec', 0.038037298989236794),
 ('koi_prad', 0.035252804830769244),
 ('koi_duration_err1', 0.02265339823941591),
 ('koi_duration', 0.020348805604299632),
 ('koi_duration_err2', 0.020170288658895214),
 ('koi_impact', 0.017331343464750387),
 ('koi_prad_err2', 0.016008810965106368),
 ('ra', 0.015249961589793994),
 ('koi_kepmag', 0.013206270168536241),
 ('koi_time0bk', 0.012516951964590102),
 ('dec', 0.012456959384930934),
 ('koi_steff_err1', 0.011794267754934274),
 ('koi_period', 0.011500573879916594),
 ('koi_steff_err2', 0.010537165069315417),
 ('koi_prad_err1', 0.009034673029845708),
 ('koi_srad_err1', 0.008979033040829988),
 ('koi_impact_err2', 0.008775394725799426),
 ('koi_time0bk_err2', 0.008402811703359315),
 ('koi_impact_err1', 0.008325531047207688),
 ('koi_time0bk_err1', 0.008263757749428616),
 ('koi_depth', 0.

In [44]:
#create a copy of dfx_important_features (most recent x values)
dfx_important_features1 = dfx_important_features.copy()
#drop columns of features with importance value of less than 1%
dfx_important_features1 = dfx_important_features1.drop(columns=['koi_teq', 'koi_insol_err2', 'koi_insol',
                                                               'koi_period_err1'])

In [45]:
#split data into testing and training data
#the random state remained at 55

x_train, x_test, y_train, y_test = train_test_split(dfx_important_features1, dfy, random_state=55)
#scale x values
x_scaler = StandardScaler().fit(x_train)
x_train = x_scaler.transform(x_train)
x_test = x_scaler.transform(x_test)

In [49]:
#use same parameters as most recent best per grid searches for model
model2 = RandomForestClassifier(bootstrap=True, max_depth=60, max_features=25, n_estimators=375)

In [50]:
#fit model
model2.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=60, max_features=25, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=375,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [51]:
model2.score(x_test, y_test)

0.8924485125858124

In [52]:
#try to increase training data
#the random state remained at 55

x_train, x_test, y_train, y_test = train_test_split(dfx_important_features1, dfy, random_state=55, train_size=0.90)
#per documentation, setting train_size also automatically effectively sets the test size


#scale x values
x_scaler = StandardScaler().fit(x_train)
x_train = x_scaler.transform(x_train)
x_test = x_scaler.transform(x_test)

In [53]:
#create model using best paramers, per most recent grid
model3 = RandomForestClassifier(bootstrap=True, max_depth=60, max_features=25, n_estimators=375)

In [54]:
model3.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=60, max_features=25, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=375,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [55]:
#with the training data increased to 90%, the r2 score increases to above 0.9
#am generally happy with this, but want to see if this can be increased
model3.score(x_test, y_test)

0.9014285714285715

In [56]:
#the training data still returns a score of 1.0/perfect, and testing is barely above .9,
#so there may still be an issue of overfitting
model3.score(x_train, y_train)

1.0

In [59]:
#try another grid search
#will keep estimators the same, and will skew towards existing optimal params based on previous models
max_depth = list(np.arange(20, 71, 10))
max_features = list(np.arange(10, 31, 5)) #need to keep the max features to be no more than 30 (n_features)
max_features.append('auto')
max_features.append('log2')
max_depth.append(None)
param_grid = {
    'bootstrap': [True], 
    'n_estimators': [375],
    'max_features': max_features,
    'max_depth': max_depth,
    'n_jobs': [-1]
}
grid = GridSearchCV(model3, param_grid, verbose=3)



In [60]:
grid.fit(x_train, y_train)

Fitting 3 folds for each of 49 candidates, totalling 147 fits
[CV] bootstrap=True, max_depth=20, max_features=10, n_estimators=375, n_jobs=-1 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  bootstrap=True, max_depth=20, max_features=10, n_estimators=375, n_jobs=-1, score=0.899, total=   3.3s
[CV] bootstrap=True, max_depth=20, max_features=10, n_estimators=375, n_jobs=-1 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    3.3s remaining:    0.0s


[CV]  bootstrap=True, max_depth=20, max_features=10, n_estimators=375, n_jobs=-1, score=0.903, total=   1.6s
[CV] bootstrap=True, max_depth=20, max_features=10, n_estimators=375, n_jobs=-1 


...

[CV]  bootstrap=True, max_depth=40, max_features=15, n_estimators=375, n_jobs=-1, score=0.899, total=   2.5s
[CV] bootstrap=True, max_depth=40, max_features=15, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=40, max_features=15, n_estimators=375, n_jobs=-1, score=0.899, total=   2.5s
[CV] bootstrap=True, max_depth=40, max_features=15, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=40, max_features=15, n_estimators=375, n_jobs=-1, score=0.893, total=   2.5s
[CV] bootstrap=True, max_depth=40, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=40, max_features=20, n_estimators=375, n_jobs=-1, score=0.899, total=   3.3s
[CV] bootstrap=True, max_depth=40, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=40, max_features=20, n_estimators=375, n_jobs=-1, score=0.900, total=   3.3s
[CV] bootstrap=True, max_depth=40, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=40, max_features=2

[CV]  bootstrap=True, max_depth=60, max_features=15, n_estimators=375, n_jobs=-1, score=0.899, total=   2.5s
[CV] bootstrap=True, max_depth=60, max_features=15, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=15, n_estimators=375, n_jobs=-1, score=0.893, total=   2.5s
[CV] bootstrap=True, max_depth=60, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=20, n_estimators=375, n_jobs=-1, score=0.897, total=   3.2s
[CV] bootstrap=True, max_depth=60, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=20, n_estimators=375, n_jobs=-1, score=0.898, total=   3.2s
[CV] bootstrap=True, max_depth=60, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=20, n_estimators=375, n_jobs=-1, score=0.895, total=   3.5s
[CV] bootstrap=True, max_depth=60, max_features=25, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=60, max_features=2

[CV]  bootstrap=True, max_depth=None, max_features=15, n_estimators=375, n_jobs=-1, score=0.895, total=   2.5s
[CV] bootstrap=True, max_depth=None, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=None, max_features=20, n_estimators=375, n_jobs=-1, score=0.899, total=   3.2s
[CV] bootstrap=True, max_depth=None, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=None, max_features=20, n_estimators=375, n_jobs=-1, score=0.898, total=   3.1s
[CV] bootstrap=True, max_depth=None, max_features=20, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=None, max_features=20, n_estimators=375, n_jobs=-1, score=0.896, total=   3.3s
[CV] bootstrap=True, max_depth=None, max_features=25, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_depth=None, max_features=25, n_estimators=375, n_jobs=-1, score=0.895, total=   4.1s
[CV] bootstrap=True, max_depth=None, max_features=25, n_estimators=375, n_jobs=-1 
[CV]  bootstrap=True, max_dept

[Parallel(n_jobs=1)]: Done 147 out of 147 | elapsed:  6.6min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=60,
                                              max_features=25,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=375, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             iid='wa

In [61]:
best_parameters2 = grid.best_params_
best_parameters2

{'bootstrap': True,
 'max_depth': 20,
 'max_features': 10,
 'n_estimators': 375,
 'n_jobs': -1}

In [62]:
model4 = RandomForestClassifier(bootstrap=True, max_depth=20, max_features=10, n_estimators=375)

In [63]:
model4.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=20, max_features=10, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=375,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [64]:
#testing score went down by just under half percent (0.004)
#don't see the score increasing from here
model4.score(x_test, y_test)

0.9

In [65]:
model4.score(x_train, y_train)

0.9928469241773963

In [66]:
file = 'random_forest.sav'
joblib.dump(model3, file)

['random_forest.sav']